In [172]:
# LIBRARIES
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process

In [173]:
def read_data(file_name, header=0, sep=','):
    """
    Reads a csv file and returns a pandas dataframe
    """
    return pd.read_csv(file_name, sep=sep, header=header)

In [174]:
def remove_columns(df, columns):
    """
    Removes columns from a dataframe
    """
    return df.drop(columns, axis=1)

In [175]:
def remove_rows(df, rows):
    """
    Removes rows from a dataframe
    """
    return df.drop(rows, axis=0)

In [176]:
def remove_duplicates(df):
    """
    Removes duplicates from a dataframe
    """
    return df.drop_duplicates()

In [177]:
def convert_to_datatype(df, column, datatype):
    """
    Converts a column to a datatype
    """
    return df[column].astype(datatype)

In [193]:
def remove_outlier_numeric(df, column, min=-np.inf, max=np.inf):
    """
    Removes rows from a dataframe based on a condition
    """
    if min != -np.inf and max != np.inf:
        return df[(df[column] >= min) & (df[column] <= max)]
    elif min == -np.inf and max != np.inf:
        return df[(df[column] <= max)]
    elif min != -np.inf and max == np.inf:
        return df[(df[column] >= min)]

In [179]:
def correct_category_levenshtein(df, column, correct_categories, threshold=80):
    """
    Corrects a column by using fuzzywuzzy to find the correct category
    """
    inconsistent_categories = pd.array(list(set(df[column].unique()) - set(correct_categories)))
    
    for inconsistent_category in inconsistent_categories:
        potential_match = process.extractOne(inconsistent_category, correct_categories)
        if potential_match[1] > threshold:
            df.loc[df[column] == inconsistent_category, column] = potential_match[0]
    return df

In [180]:
def convert_categorical_data(df, column):
    """
    Converts categorical data to numeric data
    """
    return pd.get_dummies(df[column], prefix=column)

In [181]:
def fill_missing_values(df, column, value):
    """
    Fills missing values in a column with a value
    """
    return df[column].fillna(value)

In [182]:
def drop_missing_values(df, column):
    """
    Drops missing values in a column
    """
    return df.dropna(subset=[column])

In [183]:
def knn_impute(df, column, k):
    """
    Imputation using KNN
    """
    return df[column].fillna(df[column].mode()[0])

In [184]:
def pearson_spearman_correlation(df, column, output_column):
    """
    Calculates the pearson and spearman correlation between two columns
    """
    return df[column].corr(df[output_column], method='pearson'), df[column].corr(df[output_column], method='spearman')

# TESTING FUNCTIONS

In [185]:
titanic_df = read_data('../data/raw/titanic/titanic.csv', header=0)

In [186]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",mal,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [187]:
titanic_df['Sex'].unique()

array(['mal', 'female', 'male'], dtype=object)

In [188]:
correct_categories = ['male', 'female']

titanic_df = correct_category_levenshtein(titanic_df, 'Sex', correct_categories, 80)

In [189]:
print(pearson_spearman_correlation(titanic_df, 'Fare', 'Pclass'))

(-0.5494996199439078, -0.6880316726256098)


In [194]:
print(titanic_df['Age'].describe())
titanic_df = remove_outlier_numeric(titanic_df, 'Age', 10, 70).sort_values(by='Age')
titanic_df['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64


count    647.000000
mean      31.816847
std       12.241984
min       10.000000
25%       22.000000
50%       30.000000
75%       39.000000
max       70.000000
Name: Age, dtype: float64